In [1]:
import numpy as np
import pandas as pd
from ast import literal_eval  
from nltk.stem.snowball import SnowballStemmer 
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
from collections import defaultdict
import matplotlib.pyplot as plt
import plotly.express as px
from wordcloud import WordCloud
import seaborn as sns
import networkx as nx
import json
import re
import warnings 

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits= pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [4]:
movies=movies.merge(credits,on='title')

In [5]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [6]:
movies=movies[['id','title','overview','genres','keywords','cast','crew']]

In [7]:
movies.head(1)

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [8]:
movies.isnull().sum()

id          0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [9]:
movies.dropna(inplace=True)

In [10]:
movies.isnull().sum()

id          0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [11]:
movies.duplicated().sum()

0

In [12]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [13]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L
        

In [14]:
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [15]:
convert('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')


['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [16]:
movies['genres']=movies['genres'].apply(convert)

In [17]:
movies['keywords']=movies['keywords'].apply(convert)

In [18]:
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [19]:
def convert1(obj):
    L = []
    ct = 0
    for i in ast.literal_eval(obj):
        if(ct != 3):
            L.append(i['name'])
            ct+=1
        else:
            break
    return L

In [20]:
movies['cast']=movies['cast'].apply(convert1)

In [21]:
def convert2(obj):
    L = []
    for i in ast.literal_eval(obj):
        if(i['job'] == 'Director'):
            L.append(i['name'])
            break
    return L

In [22]:
movies['crew']=movies['crew'].apply(convert2)

In [23]:
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [24]:
movies['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [25]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [26]:
movies['genres']= movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']= movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [27]:
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [28]:
movies['tags'] = movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [29]:
movies.head()

,id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[John, Carter, is, a, war-weary,, former, mili..."


In [30]:
movies=movies[['id','title','tags']]
lol=movies['tags']

In [31]:
movies.head()

,id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [32]:
movies

,id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...
4804,9367,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui..."
4805,72766,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended..."
4806,231617,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,..."
4807,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


In [33]:
movies['tags']=movies['tags'].apply(lambda x:" ".join(x))

In [34]:
movies.head()


,id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [35]:
movies['tags'].head()

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: tags, dtype: object

In [36]:
movies


,id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."
...,...,...,...
4804,9367,El Mariachi,El Mariachi just wants to play his guitar and ...
4805,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...
4806,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic..."
4807,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...


In [37]:
movies['tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [38]:
movies['tags']=movies['tags'].apply(lambda x:x.lower())

In [39]:
movies

,id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."
...,...,...,...
4804,9367,El Mariachi,el mariachi just wants to play his guitar and ...
4805,72766,Newlyweds,a newlywed couple's honeymoon is upended by th...
4806,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduces a dedic..."
4807,126186,Shanghai Calling,when ambitious new york attorney sam is sent t...


In [40]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [41]:
cv.fit_transform(movies['tags']).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [42]:
vectors  = cv.fit_transform(movies['tags']).toarray()

In [43]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [44]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [45]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [46]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [47]:
movies['tags']=movies['tags'].apply(stem)

In [48]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [49]:
from sklearn.metrics.pairwise import cosine_similarity

In [50]:
similarity = cosine_similarity(vectors)

In [51]:
similarity[0]

array([1.        , 0.08964215, 0.05976143, ..., 0.02519763, 0.02817181,
       0.        ])

In [52]:
vectors  = cv.fit_transform(movies['tags']).toarray()

In [53]:
similarity = cosine_similarity(vectors)

In [54]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:7]

[(1216, 0.28676966733820225),
 (2409, 0.26901379342448517),
 (3730, 0.2605130246476754),
 (507, 0.255608593705383),
 (539, 0.2503866978335957),
 (582, 0.24511108480187255)]

In [55]:
def recommend(movie):
    movie_list = []
    ind=movies[movies['title']==movie].index[0]
    lol=similarity[ind]
    movie_list=sorted(list(enumerate(similarity[ind])),reverse=True,key=lambda x:x[1])[1:7]
    return movie_list


In [56]:
x=recommend('Batman Begins')
print(x)

[(65, 0.40218090755486674), (1363, 0.35434169344615046), (1362, 0.3340765523905305), (3, 0.31774445465112117), (3297, 0.3120099844792576), (4148, 0.3021836115095176)]


In [57]:
movies

,id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."
...,...,...,...
4804,9367,El Mariachi,el mariachi just want to play hi guitar and ca...
4805,72766,Newlyweds,a newlyw couple' honeymoon is upend by the arr...
4806,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduc a dedic q..."
4807,126186,Shanghai Calling,when ambiti new york attorney sam is sent to s...


In [58]:
movies[movies['title']=='Iron Man 3'].index[0]

31

In [59]:
movies

,id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."
...,...,...,...
4804,9367,El Mariachi,el mariachi just want to play hi guitar and ca...
4805,72766,Newlyweds,a newlyw couple' honeymoon is upend by the arr...
4806,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduc a dedic q..."
4807,126186,Shanghai Calling,when ambiti new york attorney sam is sent to s...


In [60]:
from tkinter import *
from PIL import ImageTk,Image
from tkinter import messagebox
from tkinter import filedialog
import sqlite3
import json
import io
import requests
from ttkwidgets.autocomplete import AutocompleteCombobox
try:
	# Python2
	from urllib2 import urlopen
	import Tkinter as tk
except ImportError:
	# Python3
	from urllib.request import urlopen
	import tkinter as tk

ModuleNotFoundError: No module named 'ttkwidgets'

In [ ]:
movies_data = pd.read_csv("movies_metadata.csv", low_memory=False)
warnings.filterwarnings("ignore")
credits = pd.read_csv('credits.csv')
keywords = pd.read_csv('keywords.csv')
links_small = pd.read_csv('links_small.csv')
ratings = pd.read_csv("ratings_small.csv")
movies_data = movies_data.dropna(subset=['vote_average', 'vote_count'])

def weighted_rating(v, R):
    '''

    This function calculate weighted rating of a movies using IMDB formula

    Parameters: v (int): vote count
                R (int): vote average
    Returns: (float) IMDB score

    '''
    return ((v / (v + m)) * R) + ((m / (m + v)) * C)


C = movies_data['vote_average'].mean()  # mean vote across all data
m = movies_data['vote_count'].quantile(0.95)  # movies with more than 95% votes is taken (95 percentile)

# Taking movies whose vote count is greater than m
top_movies = movies_data.copy().loc[movies_data['vote_count'] >= m]
top_movies = top_movies.reset_index()

top_movies['score'] = ''

for i in range(top_movies.shape[0]):
    v = top_movies['vote_count'][i]  # number of vote count of the movie
    R = top_movies['vote_average'][i]  # average rating of the movie
    top_movies['score'][i] = weighted_rating(v, R)

top_movies = top_movies.sort_values('score', ascending=False)  # sorting movies in descending order according to score
top_movies = top_movies.reset_index()

# top_movies[['title', 'vote_count', 'vote_average', 'score']].head(20) # top 20 movies
t1 = top_movies[['title', 'score','id']].head(6)
def basrecrat():
    a = []
    for i in t1['id']:
        a.append(i)
    return a
    # put this inside a function to get recommendations based on the user rating



                                                    #genre based.
genres = set()

for i in range(top_movies['genres'].shape[0]):
    top_movies['genres'][i] = top_movies['genres'][i].replace("'", '"')

    # Ensure that all keys are enclosed in double quotes
    top_movies['genres'][i] = re.sub(r'([{,]\s*)(\w+)(\s*:)', r'\1"\2"\3', top_movies['genres'][i])

    # Parse the JSON string
    top_movies['genres'][i] = json.loads(top_movies['genres'][i])
    for x in top_movies['genres'][i]:
        genres.add(x['name'])

# creating map of string (genre name) and movies names(dataframe)
genres_based = dict()
for i in range(top_movies['genres'].shape[0]):
    for x in top_movies['genres'][i]:
        if x['name'] not in genres_based.keys():
            genres_based[x['name']] = pd.DataFrame(columns=top_movies.columns)
        genres_based[x['name']] = genres_based[x['name']].append(top_movies.iloc[i])

    # In[9]:

# Visualizing frequency of occurence of different genres

# Creating a count vector (list) containing frequency of a perticular genre
cnt = list()
for i in genres:
    cnt.append(genres_based[i].shape[0])

# Making a dataframe
genre_cnt = pd.DataFrame({'genres': list(genres),
                          'count': cnt

                          },
                         columns=['genres', 'count']
                         )

# fig = px.bar(genre_cnt, x='genres', y='count')
# fig.show()

# In[10]:


# print(top_movies)


# In[11]:


def genres_based_rcmnd(name):
    '''

    This function returns the top 10 movies of the given genre

    Parameters: name (string): Name of the genre

    Returns: (Dataframe) Top 10 move recommendation

    '''

    if name not in genres:
        return None
    else:
        return genres_based[name][['title', 'vote_count', 'vote_average', 'score','id']].head(6)


# print(genres_based_rcmnd("War"))
def basrecgen(name):
    b = []
    for i in genres_based_rcmnd(name)['id']:
        b.append(i)
    return b
#put this inside a function to get the recommendations based on the genres. for the final cut.

In [ ]:
def cop():
    a = ['745','318','926','2973','1262','308']
    return a

In [ ]:
root = Tk()
root.title("MSG BOX")
root.geometry("1920x1080")

def az():
	pq=Toplevel()
	pq.geometry("1920x1080")
	bg1 = Image.open('download.jpeg')
	bg2=bg1.resize((1920,1080))
	bg3=ImageTk.PhotoImage(bg2)
	label2 = Label(pq,image = bg3)
	label2.image=bg3
	label2.place(x=0,y=0)
    
def op():
	def tel():
		x=clicked.get()
		if clicked.get() == 'Arateaur of movies try this':
			def rec():
				st=entry.get()
				x=recommend(st)
				b = []
				img = []
				for i in x:
					b.append(movies.iloc[i[0]].title)
					part1 = "https://image.tmdb.org/t/p/w500"
					part2 = pos(movies.iloc[i[0]].id)
					url = part1 + part2
					print(url)
					fin = urlopen(url)
					s = io.BytesIO(fin.read())
					pil_image = Image.open(s)
					tk_image = ImageTk.PhotoImage(pil_image)
					img.append(tk_image)
				top1=Toplevel()
				global my_l
				my_l = Label(top1,image=img[0])
				my_l.grid(row=0,column=0,columnspan=3)
				def forward(image_number):
					global my_l
					global button_forward
					global button_back
					my_l.grid_forget()
					my_l=Label(top1,image=img[image_number])
					button_forward = Button(top1,text = '>>',command=lambda: forward(image_number+1))
					button_back = Button(top1,text = '<<',command=lambda: back(image_number-1))
					if image_number == 5:
						button_forward=Button(top1,text='>>',state=DISABLED)
					my_l.grid(row=0,column=0,columnspan=3)
					button_back.grid(row=1,column=0)
					button_forward.grid(row=1,column=2)
				def back(image_number):
					global my_l
					global button_forward
					global button_back
					my_l.grid_forget()
					my_l=Label(top1,image=img[image_number])
					button_forward = Button(top1,text = '>>',command=lambda: forward(image_number+1))
					button_back = Button(top1,text = '<<',command=lambda: back(image_number-1))
					if image_number == 0:
						button_back=Button(top1,text='<<',state=DISABLED)
					my_l.grid(row=0,column=0,columnspan=3)
					button_back.grid(row=1,column=0)
					button_forward.grid(row=1,column=2)
				button_back=Button(top1,text='<<',state=DISABLED)
				button_exit=Button(top1,text='Exit',command=top1.quit)
				button_forward=Button(top1,text='>>',command=lambda:forward(1))
				button_back.grid(row=1,column=0)
				button_forward.grid(row=1,column=2)
				button_exit.grid(row=1,column=1)
			top=Toplevel()                
			top.title("Movies List")
			top.geometry("1920x1080")
			bg1 = ImageTk.PhotoImage(Image.open('p_3.jpg'))
			label2 = Label(top,image = bg1)
			label2.image=bg1
			label2.place(x=0,y=0)
			label4 = Label(top, text = "Enter The Movie Name",fg='Red',font=('Times', 40)).pack(pady=90)  
			entry= Entry(top,width=35, font=('Georgia 20'))
			entry.focus_set()
			entry.pack(pady=150)
			btn2=Button(top,text="Search",command=rec).pack()                
		elif clicked.get() == 'New to movies try this':
			def rec():
				st=entry1.get()
				x=basrecgen(st)
				b = []
				img = []
				for i in x:
					part1 = "https://image.tmdb.org/t/p/w500"
					part2 = pos(int(i))
					url = part1 + part2
					print(url)
					fin = urlopen(url)
					s = io.BytesIO(fin.read())
					pil_image = Image.open(s)
					tk_image = ImageTk.PhotoImage(pil_image)
					img.append(tk_image)
				top1=Toplevel()
				global my_l
				my_l = Label(top1,image=img[0])
				my_l.grid(row=0,column=0,columnspan=3)
				def forward(image_number):
					global my_l
					global button_forward
					global button_back
					my_l.grid_forget()
					my_l=Label(top1,image=img[image_number])
					button_forward = Button(top1,text = '>>',command=lambda: forward(image_number+1))
					button_back = Button(top1,text = '<<',command=lambda: back(image_number-1))
					if image_number == 5:
						button_forward=Button(top1,text='>>',state=DISABLED)
					my_l.grid(row=0,column=0,columnspan=3)
					button_back.grid(row=1,column=0)
					button_forward.grid(row=1,column=2)
				def back(image_number):
					global my_l
					global button_forward
					global button_back
					my_l.grid_forget()
					my_l=Label(top1,image=img[image_number])
					button_forward = Button(top1,text = '>>',command=lambda: forward(image_number+1))
					button_back = Button(top1,text = '<<',command=lambda: back(image_number-1))
					if image_number == 0:
						button_back=Button(top1,text='<<',state=DISABLED)
					my_l.grid(row=0,column=0,columnspan=3)
					button_back.grid(row=1,column=0)
					button_forward.grid(row=1,column=2)
				button_back=Button(top1,text='<<',state=DISABLED)
				button_exit=Button(top1,text='Exit',command=top1.quit)
				button_forward=Button(top1,text='>>',command=lambda:forward(1))
				button_back.grid(row=1,column=0)
				button_forward.grid(row=1,column=2)
				button_exit.grid(row=1,column=1)
			def lol():
				x=basrecrat()
				b = []
				img = []
				for i in x:
					part1 = "https://image.tmdb.org/t/p/w500"
					part2 = pos(int(i))
					url = part1 + part2
					print(url)
					fin = urlopen(url)
					s = io.BytesIO(fin.read())
					pil_image = Image.open(s)
					tk_image = ImageTk.PhotoImage(pil_image)
					img.append(tk_image)
				top1=Toplevel()
				global my_l
				my_l = Label(top1,image=img[0])
				my_l.grid(row=0,column=0,columnspan=3)
				def forward(image_number):
					global my_l
					global button_forward
					global button_back
					my_l.grid_forget()
					my_l=Label(top1,image=img[image_number])
					button_forward = Button(top1,text = '>>',command=lambda: forward(image_number+1))
					button_back = Button(top1,text = '<<',command=lambda: back(image_number-1))
					if image_number == 5:
						button_forward=Button(top1,text='>>',state=DISABLED)
					my_l.grid(row=0,column=0,columnspan=3)
					button_back.grid(row=1,column=0)
					button_forward.grid(row=1,column=2)
				def back(image_number):
					global my_l
					global button_forward
					global button_back
					my_l.grid_forget()
					my_l=Label(top1,image=img[image_number])
					button_forward = Button(top1,text = '>>',command=lambda: forward(image_number+1))
					button_back = Button(top1,text = '<<',command=lambda: back(image_number-1))
					if image_number == 0:
						button_back=Button(top1,text='<<',state=DISABLED)
					my_l.grid(row=0,column=0,columnspan=3)
					button_back.grid(row=1,column=0)
					button_forward.grid(row=1,column=2)
				button_back=Button(top1,text='<<',state=DISABLED)
				button_exit=Button(top1,text='Exit',command=top1.quit)
				button_forward=Button(top1,text='>>',command=lambda:forward(1))
				button_back.grid(row=1,column=0)
				button_forward.grid(row=1,column=2)
				button_exit.grid(row=1,column=1)
			top=Toplevel()
			top.title("Movies List")
			top.geometry("1920x1080")
			bg1 = Image.open('ll.jpg')
			bg2=bg1.resize((1920,1080))
			bg3=ImageTk.PhotoImage(bg2)
			label2 = Label(top,image = bg3)
			label2.image=bg3
			label2.place(x=0,y=0)
			label4 = Label(top, text = "Enter The Genere of the Movie",fg='Red',font=('Times', 40)).pack(pady=90)  
			entry1= Entry(top,width=35, font=('Georgia 20'))
			entry1.focus_set()
			entry1.pack(pady=150)
			btn3=Button(top,text="Wanna see the best movie",fg='Red',font=('Times', 27),command=lol).pack()  
			btn2=Button(top,text="Search",command=rec).pack(pady=20)
            
		else:
			def lol():
				x=cop()
				b = []
				img = []
				for i in x:
					part1 = "https://image.tmdb.org/t/p/w500"
					part2 = pos(int(i))
					url = part1 + part2
					print(url)
					fin = urlopen(url)
					s = io.BytesIO(fin.read())
					pil_image = Image.open(s)
					tk_image = ImageTk.PhotoImage(pil_image)
					img.append(tk_image)
				top1=Toplevel()
				global my_l
				my_l = Label(top1,image=img[0])
				my_l.grid(row=0,column=0,columnspan=3)
				def forward(image_number):
					global my_l
					global button_forward
					global button_back
					my_l.grid_forget()
					my_l=Label(top1,image=img[image_number])
					button_forward = Button(top1,text = '>>',command=lambda: forward(image_number+1))
					button_back = Button(top1,text = '<<',command=lambda: back(image_number-1))
					if image_number == 5:
						button_forward=Button(top1,text='>>',state=DISABLED)
					my_l.grid(row=0,column=0,columnspan=3)
					button_back.grid(row=1,column=0)
					button_forward.grid(row=1,column=2)
				def back(image_number):
					global my_l
					global button_forward
					global button_back
					my_l.grid_forget()
					my_l=Label(top1,image=img[image_number])
					button_forward = Button(top1,text = '>>',command=lambda: forward(image_number+1))
					button_back = Button(top1,text = '<<',command=lambda: back(image_number-1))
					if image_number == 0:
						button_back=Button(top1,text='<<',state=DISABLED)
					my_l.grid(row=0,column=0,columnspan=3)
					button_back.grid(row=1,column=0)
					button_forward.grid(row=1,column=2)
				button_back=Button(top1,text='<<',state=DISABLED)
				button_exit=Button(top1,text='Exit',command=top1.quit)
				button_forward=Button(top1,text='>>',command=lambda:forward(1))
				button_back.grid(row=1,column=0)
				button_forward.grid(row=1,column=2)
				button_exit.grid(row=1,column=1)
			top=Toplevel()
			top.title("Movies List")
			top.geometry("1920x1080")
			bg1 = Image.open('aa.jpg')
			bg2=bg1.resize((1920,1080))
			bg3=ImageTk.PhotoImage(bg2)
			label2 = Label(top,image = bg3)
			label2.image=bg3
			label2.place(x=0,y=0) 
			btn2=Button(top,text="Let's see how your mindset guys watching the movies",bg='black',fg='red',font=('Times',30),command=lol).place(x=500,y=300)
	ps=Toplevel()
	ps.geometry("1920x1080")
	bg4 = ImageTk.PhotoImage(Image.open('p_5.jpg'))
	label5 = Label(ps,image = bg4)
	label5.image=bg4
	label5.place(x=0,y=0)
	label6 = Label(ps, text = "You can see the Movie now\n by Clicking it",fg='Yellow',bg='black',font=('italic', 30)).pack(pady=90)
	clicked=StringVar()
	clicked.set("Let me Show your skills in Movies")
	drop=OptionMenu(ps,clicked,"New to movies try this","Arateaur of movies try this","Veteran of movies try this")
	drop.config(width=30,height=4,bg='black',fg='Yellow')
	drop.pack()
	btn3=Button(ps,text="let's check!!",bg='black',fg='red',font=('Times', 20),command=tel).pack(pady=20)    


	def pos(id):
		res = requests.get(f"https://api.themoviedb.org/3/movie/{id}?api_key=774f3e728d5d12a560b88198dd8ca1a6&language=en-US")
		data =res.json()
		return data['poster_path']

    



bg = ImageTk.PhotoImage(Image.open("p_2.jpg"))
label1 = Label( root, image = bg)
label1.place(x=0,y=0)

label2 = Label( root, text = "Movie\n Recommender System",bg='black',fg='white',font=('Times', 40))
label2.place(x=800,y=100)

label3 = Label( root, text = "if(cp):",fg='black',font=('Times', 40)).place(x=800,y=300)
btn2=Button(root,text="Go ahead and do cp",bg='black',fg='red',font=('Times', 30),command=az).place(x=850,y=370)
label4 = Label( root, text = "else:",fg='black',font=('Times', 40)).place(x=800,y=440)
btn1=Button(root,text="Recommend me movies!!",bg='black',fg='red',font=('Times', 30),command=op).place(x=850,y=510)

root.mainloop()

In [ ]:
movies
